---
- title: "'CS61A: Iterators'"
- author: alex
- badges: true
- comments: true
- categories: [CS61A]
- date: 2024-10-04 6:00:00 -0800
- math: true
- tags: [CS61A, iterators]
---

# Iterators
- Most forms of sequential data are implicity represented with an iterator.
- Iterators are a programming interface that are used in python to access the elements of various containers.
- A container provides an iterator that provides access to its element in some order
    - `iter(iterable)`
        - Returns an iterator over the elememts of the iterable.
    - `next(iterator)`
        - Returns the next element in an iterator
- The iterator in python knows the contents of a sequence, has a pointer to the current value, and also has a marker for what's next in the list.
    - Iterators basically mark out a position within the list.
    - Provides access to the element within that position, and everything after it.
    

In [10]:
s = [3, 4, 5]
t = iter(s)
print("first call to next(t)", next(t))
print("second call to next(t)", next(t))
u = iter(s)
print("first call to next(u):", next(u))
print("third call to next(t):", next(t))

first call to next(t) 3
second call to next(t) 4
first call to next(u): 3
third call to next(t): 5


- A list is not an iterator, but we may create an iterator for a list
- An iterator only stores what is left to iterate over.


In [13]:
s = [1,2,3,4,5]
t = iter(s)
next(t)
next(t)
print(list(t)) # Only [3,4,5] have not been accessed yet, they are the only values remaining in the iterator.
next(t) # Calling list on the remaining values in the iterator will also "iterate" over those values, we get an error on this line

[3, 4, 5]


StopIteration: 

## Dictionary Iteration
- We may get different views of objects in a dictionary.
- *iterables* are any values that can be passed to `iter()` to produce an iterator
- *iterators* are values returned from *iter* that may be passed to *next*
    - All iterators are mutable
    - Calling `next()` or some other function that access element(s) from the iterator, will mutate the iterator to point to the next unaccessed element.
- The dictionary, its keys, its values, and its items are all iterable values.
    - Order of items within a dictionary is the order in which they are added (Python 3.6+)
    - Historically, items appeared in an arbitrary order (Python 3.5 and earlier)


In [17]:
d = {'one': 1, 'two': 2, 'three': 3}
d['zero'] = 0
k = iter(d.keys()) # or iter(d)
print(next(k))
print(next(k))
print(next(k))
print(next(k))

one
two
three
zero


In [18]:
v = iter(d.values())
print(next(v))
print(next(v))
print(next(v))
print(next(v))

1
2
3
0


In [19]:
i = iter(d.items()) # items within dictionaries represented as key-value pairs of tuples
print(next(i))
print(next(i))
print(next(i))
print(next(i))

('one', 1)
('two', 2)
('three', 3)
('zero', 0)


- If we make any additions or removals to the dictionary size while an iterator is active, that iterator would become invalid because the dictionary had been mutated.

## For Statements
- Iterates over iterable values and iterators.
    - For iterators, the for loop increments the marker of the iterator all the way to the end of the sequence.
    - Once we reach the end of an iterator, nothing would be returned by the iterator.
    - We can iterate through an iterable object as many times as I'd like.

In [5]:
ri = iter(range(3, 6))
for i in ri:
    print(i)

for i in ri: # Nothing gets printed this loop
    print(i)

3
4
5


# Built-In Iterator Functions
- These functions take in an iterable and returns an iterator.
- Most built-in Python sequence operations return iterators that compute results lazily.
    - This means that not everything within the sequence is computed and stored in memory. The values are computed the instant that the value is being accessed.

## Map function
- `map(func, iterable)` 
    - Applies the function to every element in the iterable.
    - Does not apply the function immediately, rather, an iterator is returned that iterates over every `func(x)`.
- `filter(func, iterable)`
    - The returned iterable iterates every over every x in the iterable so long as `func(x)` returns true.
- `zip(first_iter, second_iter)`
    - Iterates over co-indexed pairs of elements from both iterables
- `reversed(sequence)`
    - Iterates over x in a sequence in reverse order
    

In [12]:
i1 = iter([1,2,3,4,5,6])
i2 = iter([2,4,6,8,10,12])
for x, y in zip(i1, i2):
    print(y-x)

1
2
3
4
5
6


- To view the (remaining) contents of an iterator, place the resulting elements into a container
    - `list(iterable)`
        - Creates a list containing all x left in iterable.
    - `tuple(iterable)`
        - Creates a tuple containing all x left in iterable.
    - `sorted(iterable)`
        - Creates a sorted list containing all x left in iterable.
- Example 1:

In [16]:
def double(x):
    print('**', x, '=>', 2*x, '**')
    return 2*x

m = map(double, [1,2,3,4]) # This is lazy construction
print(next(m)) # Each value within the map is not calculated until it is invoked.
print(next(m))
print(next(m))
print(next(m))

** 1 => 2 **
2
** 2 => 4 **
4
** 3 => 6 **
6
** 4 => 8 **
8


- Example 2:
    - In this case, the filter iterator did just as much work that it needed to do to return the first valid value (2*5 = 10 >= 10), and stops computation there.

In [21]:
m = map(double, range(3, 7))
f = lambda y: y >= 10
t = filter(f, m)
print(next(t))

** 3 => 6 **
** 4 => 8 **
** 5 => 10 **
10


- Invoking list or any other container constructor on these iterators will immediately exahust all remaining values within that iterator.
- This type of lazy computation saves a lot of time as we do not always calculate values that we do not need.
- Do not ever compare equality or identity between a list and an iterator, it will return `False`.

## The Zip Function
- The built-in `zip` function returns an iterator over tuples of co-indexed pairs from each iterable.

In [23]:
list(zip([1,2], [3,4]))

[(1, 3), (2, 4)]

- If one iterable happens to be longer than the other, `zip` only iterates over the matched indicies and skips any extra elements.
- More than two iterables may be passed to `zip`.

In [24]:
list(zip([1,2,3],[2,3,4],[3,4,5]))

[(1, 2, 3), (2, 3, 4), (3, 4, 5)]

- Example: Implement **palindrome**, to check whether `s` is the same forward and backward.

In [27]:
def palindrome(s):
    return all([a == b for a, b in zip(s, reversed(s))])

palindrome("racecar")

True

## Reasons for Using Iterators
- When code is written to process iterators, the code makes few assumptions about the data.
    - We do not assume the type of the data
    - Thus, when we change representation from a list to a tuple, map object, or dict_keys does not require rewriting our existing code.
- Iterators also bundle a sequence and the position within the sequence as one object.
    - Passing the iterator to other functions wil always retain the current position within the iterator.
    - We may ensure that each element of the sequence is processed only once.
- Passing iterators around limits the operations that may be performed on the sequence to only using the `next()` function.

